Import packages

In [5]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import os

Import GPA Data

In [3]:
# File name
filename = "../Data/GPA_Data/gas_station_abu_dhabi.csv"

# Read the CSV file
df = pd.read_csv(filename)

# Preview the first 2 rows
df.head(2)

,name,id,types,nationalPhoneNumber,internationalPhoneNumber,formattedAddress,addressComponents,plusCode_globalCode,location_latitude,location_longitude,...,servesLunch,servesDinner,liveMusic,servesDessert,servesCoffee,goodForGroups,priceRange_startPrice_currencyCode,priceRange_startPrice_units,priceRange_endPrice_currencyCode,priceRange_endPrice_units
0,places/ChIJWd_MDxo9Zz4RTy6Tw7rvZ3c,ChIJWd_MDxo9Zz4RTy6Tw7rvZ3c,"gas_station, point_of_interest, establishment",800 300,+971 800 300,Unnamed Road - Al 'Azeezah - New Seih Al Khair...,"[{""longText"": ""Abu Dhabi"", ""shortText"": ""Abu D...",7HMM9Q5C+8X,23.358356,53.772422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,places/ChIJh_LxqkiHXT4RK_JdIm1YI34,ChIJh_LxqkiHXT4RK_JdIm1YI34,"gas_station, point_of_interest, establishment",NaN,NaN,Habshan - Abu Dhabi - United Arab Emirates,"[{""longText"": ""Habshan"", ""shortText"": ""Habshan...",7HMMPGQG+M4,23.739190,53.525324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load Abu Dhabi Shapefile (Admin Level 3)

In [8]:
# Load the shapefile using GeoPandas
data_folder = "../Data/UAE_Shapefile/gadm41_ARE_shp"
shapefile_name = "gadm41_ARE_3.shp"
shapefile_path = os.path.join(data_folder, shapefile_name)
#print("Shapefile path => ", shapefile_path)
gdf = gpd.read_file(shapefile_path)

# Filter for Abu Dhabi City
abu_dhabi_gdf = gdf[gdf['NAME_2'] == 'Abu Dhabi']

# Remove "Sir Abu Nu'Ayr" district
abu_dhabi_gdf = abu_dhabi_gdf[abu_dhabi_gdf['NAME_3'] != "Sir Abu Nu'Ayr"]

Group by "displayName_text" {'ADNOC', 'ENOC', 'Others'}

In [10]:
df["displayName_text"].value_counts()

displayName_text
ADNOC Service Station                          9
ADNOC Service station                          2
ADNOC Service Station | Seih al Khair (727)    1
ADNOC Xpress | Al Qurm (139)                   1
محطة مدينة زايد الرياضية                       1
                                              ..
ENOC 1076                                      1
ADNOC Fuel Station                             1
ADNOC 155                                      1
ADNOC Service Station | Al Riyadh (319)        1
ENOC Compact Station | 1903 | Dubai south      1
Name: count, Length: 252, dtype: int64

In [11]:
# ADNOC: include English, Arabic, loose patterns
regex_adnoc = r'(?i)\b(adnoc|ادنوك|محطة\s*ادنوك|adnoc\s*(service\s*station|lube|cng|vehicle inspection)|aldhfra petrol pump adnoc)\b'

# ENOC: include English and Arabic potential
regex_enoc = r'(?i)\b(enoc|اينوك|محطة\s*اينوك)\b'

# ADNOC DataFrame
df_adnoc = df[df['displayName_text'].str.contains(regex_adnoc, na=False)]

# ENOC DataFrame
df_enoc = df[df['displayName_text'].str.contains(regex_enoc, na=False)]

# Others DataFrame
matched_indexes = pd.concat([df_adnoc, df_enoc]).index
df_others = df[~df.index.isin(matched_indexes)]

C:\Users\Oussama Raji\AppData\Local\Temp\ipykernel_41796\73903963.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_adnoc = df[df['displayName_text'].str.contains(regex_adnoc, na=False)]
C:\Users\Oussama Raji\AppData\Local\Temp\ipykernel_41796\73903963.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_enoc = df[df['displayName_text'].str.contains(regex_enoc, na=False)]


In [12]:
df_others['displayName_text'].value_counts()

displayName_text
BIDA AL OSHANA                                                    1
Auto Cafe                                                         1
Liberty CarCare                                                   1
Cosmo Oil Co Limited                                              1
غسيل الي لسيارات                                                  1
Snamprogetti SPA                                                  1
محطة بترول                                                        1
Abudhabi                                                          1
Al Mushrif abudhabi                                               1
محطة مدينة زايد الرياضية                                          1
AJ 474 AL RAHA BEACH SERVICES                                     1
Yas petrol station( Diesel Available)                             1
Diesel fuel                                                       1
محطة بترول الشامخة                                                1
شيشه بترول                     

Calculate the Midpoint Coordinates

In [14]:
# total_bounds → (minx, miny, maxx, maxy) for the entire GeoDataFrame
minx, miny, maxx, maxy = abu_dhabi_gdf.total_bounds

# x corresponds to longitude, y corresponds to latitude
sw_lat = miny  # southernmost lat
sw_lon = minx  # westernmost lon
ne_lat = maxy  # northernmost lat
ne_lon = maxx  # easternmost lon

# Calculate midpoint coordinates
mid_lat = (sw_lat + ne_lat) / 2
mid_lon = (sw_lon + ne_lon) / 2

Create the Map

In [16]:
# Create map
m = folium.Map(location=[mid_lat, mid_lon], zoom_start= 9, tiles='cartodbpositron')

# Add the GeoJSON polygon to the map
folium.GeoJson(
    data=abu_dhabi_gdf,
    name='Abu Dhabi Boundary',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_3'],
        aliases=['Name: '],
        localize=True
    )
).add_to(m)

# Dictionary of DataFrames and their display names/colors
df_map = {
    "ADNOC": {"df": df_adnoc, "color": "orange"},
    "ENOC": {"df": df_enoc, "color": "green"},
    "Others": {"df": df_others, "color": "red"},
}

# Add each DataFrame to the map with its own colored cluster
for brand, props in df_map.items():
    df = props["df"]
    color = props["color"]

    # Feature group for layer control
    feature_group = folium.FeatureGroup(name=brand, show=True).add_to(m)

    # Custom cluster icon style (color-matched)
    marker_cluster = MarkerCluster(
        icon_create_function=f"""
        function(cluster) {{
            return L.divIcon({{
                html: '<div style="background-color: {color}; color: white; border-radius: 50%; width: 30px; height: 30px; line-height: 30px; text-align: center; font-weight: bold; font-size: 12px;">' + cluster.getChildCount() + '</div>',
                className: 'marker-cluster',
                iconSize: new L.Point(30, 30)
            }});
        }}
        """
    ).add_to(feature_group)

    for _, row in df.iterrows():
        latitude = row['location_latitude']
        longitude = row['location_longitude']
        name = row['displayName_text']
        address = row.get('formattedAddress', 'No Address')
        rating = row.get('rating', 'No Ratings')
        link = row.get('googleMapsUri', 'No Link Provided')

        popup_content = f"""
            <b>Name:</b> {name}<br>
            <b>Address:</b> {address}<br>
            <b>Longitude:</b> {longitude}<br>
            <b>Latitude:</b> {latitude}<br>
            <b>Rating:</b> {rating}<br>
            <b>Link:</b> <a href="{link}" target="_blank">View on Google Maps</a><br>
        """

        # Colored dot
        folium.CircleMarker(
            location=[latitude, longitude],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=folium.Popup(popup_content, max_width=300)
        ).add_to(marker_cluster)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Legend
legend_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Legend</b><br>
'''

for brand, props in df_map.items():
    legend_html += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        border-radius: 50%;
        background-color: {props['color']};">
    </span> {brand}<br>
    '''

legend_html += '</div>'

# Add Legend
m.get_root().html.add_child(folium.Element(legend_html))


# Save map
m.save("../Data/Maps/Gas_Stations_Abu_Dhabi_City.html")
print(" Gas Station Map saved")

m


 Gas Station Map saved
